<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

# Assignment: Notebook for Peer Assignment

Estimated time needed: 45 minutes


# Assignment Scenario

Congratulations! You have just been hired by a US Venture Capital firm as a data analyst.

The company is considering foreign grain markets to help meet its supply chain requirements for its recent investments in the microbrewery and microdistillery industry, which is involved with the production and distribution of craft beers and spirits.

Your first task is to provide a high level analysis of crop production in Canada. Your stakeholders want to understand the current and historical performance of certain crop types in terms of supply and price volatility. For now they are mainly interested in a macro-view of Canada's crop farming industry, and how it relates to the relative value of the Canadian and US dollars.


# Introduction

Using this R notebook you will:

1.  Understand four datasets
2.  Load the datasets into four separate tables in a Db2 database
3.  Execute SQL queries unsing the RODBC R package to answer assignment questions

You have already encountered two of these datasets in the previous practice lab. You will be able to reuse much of the work you did there to prepare your database tables for executing SQL queries.


# Understand the datasets

To complete the assignment problems in this notebook you will be using subsetted snapshots of two datasets from Statistics Canada, and one from the Bank of Canada. The links to the prepared datasets are provided in the next section; the interested student can explore the landing pages for the source datasets as follows:

1.  <a href="https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMRP0203ENSkillsNetwork23863830-2021-01-01&pid=3210035901">Canadian Principal Crops (Data & Metadata)</a>
2.  <a href="https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMRP0203ENSkillsNetwork23863830-2021-01-01&pid=3210007701">Farm product prices (Data & Metadata)</a>
3.  <a href="https://www.bankofcanada.ca/rates/exchange/daily-exchange-rates?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMRP0203ENSkillsNetwork23863830-2021-01-01">Bank of Canada daily average exchange rates</a>

### 1. Canadian Principal Crops Data \*

This dataset contains agricultural production measures for the principle crops grown in Canada, including a breakdown by province and teritory, for each year from 1908 to 2020.

For this assignment you will use a preprocessed snapshot of this dataset (see below).

A detailed description of this dataset can be obtained from the StatsCan Data Portal at:
https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3210035901\
Detailed information is included in the metadata file and as header text in the data file, which can be downloaded - look for the 'download options' link.

### 2. Farm product prices

This dataset contains monthly average farm product prices for Canadian crops and livestock by province and teritory, from 1980 to 2020 (or 'last year', whichever is greatest).

For this assignment you will use a preprocessed snapshot of this dataset (see below).

A description of this dataset can be obtained from the StatsCan Data Portal at:
https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=3210007701
The information is included in the metadata file, which can be downloaded - look for the 'download options' link.

### 3. Bank of Canada daily average exchange rates \*

This dataset contains the daily average exchange rates for multiple foreign currencies. Exchange rates are expressed as 1 unit of the foreign currency converted into Canadian dollars. It includes only the latest four years of data, and the rates are published once each business day by 16:30 ET.

For this assignment you will use a snapshot of this dataset with only the USD-CAD exchange rates included (see next section). We have also prepared a monthly averaged version which you will be using below.

A brief description of this dataset and the original dataset can be obtained from the Bank of Canada Data Portal at:
https://www.bankofcanada.ca/rates/exchange/daily-exchange-rates/

( \* these datasets are the same as the ones you used in the practice lab)


### Dataset URLs

1.  Annual Crop Data: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Annual_Crop_Data.csv

2.  Farm product prices: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_Farm_Prices.csv

3.  Daily FX Data: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Daily_FX.csv

4.  Monthly FX Data: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_FX.csv

<span style="color:red">**IMPORTANT:**</span> You will be loading these datasets directly into R data frames from these URLs instead of from the StatsCan and Bank of Canada portals. The versions provided at these URLs are simplified and subsetted versions of the original datasets.


#### Now let's load these datasets into four separate Db2 tables.

Let's first load the RODBC package:


In [ ]:
library(RODBC)
install.packages("languageserver")
library(languageserver)

## Problem 1

#### Create tables

Establish a connection to the Db2 database, and create the following four tables using the RODBC package in R.
Use the separate cells provided below to create each of your tables.

1.  **CROP_DATA**
2.  **FARM_PRICES**
3.  **DAILY_FX**
4.  **MONTHLY_FX**

The previous practice lab will help you accomplish this.


### Solution 1


In [ ]:
# Establish database connection
dsn_driver <- "{IBM DB2 ODBC Driver}"
dsn_database <- "bludb"
dsn_hostname <- "0c77d6f2-5da9-48a9-81f8-86b520b87518.bs2io90l08kqb1od8lcg.databases.appdomain.cloud"
dsn_port <- "31198"
dsn_protocol <- "TCPIP"
dsn_uid <- "rqm92944"
dsn_pwd <- "nQVZUfaU4wpV0ZWB"
dsn_security <- "ssl"


conn_path <- paste("DRIVER=", dsn_driver,
";DATABASE=", dsn_database,
";HOSTNAME=", dsn_hostname,
";PORT=", dsn_port,
";PROTOCOL=", dsn_protocol,
";UID=", dsn_uid,
";PWD=", dsn_pwd, 
";SECURITY=", dsn_security, 
sep="")

conn <- odbcDriverConnect(conn_path, believeNRows=FALSE)
conn




In [ ]:
# CROP_DATA:
crop_data <- sqlQuery(conn, "CREATE TABLE CROP_DATA (CROP_ID INTEGER NOT NULL, CROP_YEAR DATE NOT NULL, CROP_TYPE VARCHAR(20) NOT NULL, GEO VARCHAR(20) NOT NULL, SEEDED_AREA INTEGER NOT NULL, HARVESTED_AREA INTEGER NOT NULL, PRODUCTION INTEGER NOT NULL, AVG_YIELD INTEGER NOT NULL PRIMARY KEY (CROP_ID))",
ERRORS=FALSE
)

    if (crop_data == -1){
        cat ("An error has occurred.\n")
        msg <- odbcGetErrMsg(conn)
        print (msg)
    } else {
        cat ("Table was created successfully.\n")
    }

In [ ]:
# FARM_PRICES:
farm_prices <- sqlQuery(conn, "CREATE TABLE FARM_PRICES (PRICE_ID INTEGER NOT NULL, PRICE_DATE DATE NOT NULL, CROP_TYPE VARCHAR(20) NOT NULL, GEO VARCHAR(50) NOT NULL, PRICE NUMERIC NOT NULL PRIMARY KEY (PRICE_ID))",
ERRORS=FALSE
)

    if (farm_prices == -1){
        cat ("An error has occurred.\n")
        msg <- odbcGetErrMsg(conn)
        print (msg)
    } else {
        cat ("Table was created successfully.\n")
    }

In [ ]:
# DAILY_FX:
daily_fx <- sqlQuery(conn, "CREATE TABLE DAILY_FX (DFX_ID INTEGER NOT NULL, DFX_DATE DATE NOT NULL, FXUSDCAD FLOAT(6), PRIMARY KEY (DFX_ID))",
errors=FALSE
)

    if (daily_fx == -1){
        cat ("An error has occurred.\n")
        msg <- odbcGetErrMsg(conn)
        print (msg)
    } else {
        cat ("Table was created successfully.\n")
    }

In [ ]:
# MONTHLY_FX:
monthly_fx <- sqlQuery(conn, "CREATE TABLE MONTHLY_FX (
                                MFX_ID INTEGER NOT NULL,
                                MFX_DATE DATE NOT NULL, 
                                FXUSDCAD FLOAT(6),
                                PRIMARY KEY (MFX_ID))",
                    errors=FALSE
                    )

    if (monthly_fx == -1){
        cat ("An error has occurred.\n")
        msg <- odbcGetErrMsg(conn)
        print (msg)
    } else {
        cat ("Table was created successfully.\n")
    }

## Problem 2

#### Read Datasets and Load Tables

Read the datasets into R dataframes using the urls provided above. Then load your tables.


### Solution 2


In [10]:
crop_data_df <- read.csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Annual_Crop_Data.csv")
farm_prices_df <- read.csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_Farm_Prices.csv")
daily_fx_df <- read.csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Daily_FX.csv")
monthly_fx_df <- read.csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-RP0203EN-SkillsNetwork/labs/Final%20Project/Monthly_FX.csv")


sqlSave(conn, crop_data_df, "CROP_DATA", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE)
sqlSave(conn, farm_prices_df, "FARM_PRICES", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE)
sqlSave(conn, daily_fx_df, "DAILY_FX", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE)
sqlSave(conn, monthly_fx_df, "MONTHLY_FX", append=TRUE, fast=FALSE, rownames=FALSE, colnames=FALSE, verbose=FALSE)

ERROR: Error in dimnames(x) <- dn: length of 'dimnames' [2] not equal to array extent


## Now execute SQL queries using the RODBC R package to solve the assignment problems.

## Problem 3

#### How many records are in the farm prices dataset?


### Solution 3


In [9]:
check_farm <- "SELECT * FROM FARM_PRICES"
check <- sqlQuery(conn, check_farm)
check

check_crop <- "SELECT * FROM CROP_DATA"
check <- sqlQuery(conn, check_crop)
check



,rownames,index,date,cropType,GEO,pricePerMT
,<int>,<int>,<fct>,<fct>,<fct>,<dbl>
1,1,0,1985-01-01,Barley,Alberta,127.39
2,2,1,1985-01-01,Barley,Saskatchewan,121.38
3,3,2,1985-01-01,Canola,Alberta,342.00
4,4,3,1985-01-01,Canola,Saskatchewan,339.82
5,5,4,1985-01-01,Rye,Alberta,100.77
6,6,5,1985-01-01,Rye,Saskatchewan,109.75
7,7,6,1985-02-01,Barley,Alberta,127.36
8,8,7,1985-02-01,Barley,Saskatchewan,119.64
9,9,8,1985-02-01,Canola,Alberta,347.27


,rownames,index,YEAR,cropType,GEO,seededArea,harvestedArea,production,avgYield
,<int>,<int>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>
1,1,0,1965-12-31,Barley,Alberta,1372000,1372000,2504000,1825
2,2,1,1965-12-31,Barley,Canada,2476800,2476800,4752900,1920
3,3,2,1965-12-31,Barley,Saskatchewan,708000,708000,1415000,2000
4,4,3,1965-12-31,Canola,Alberta,297400,297400,215500,725
5,5,4,1965-12-31,Canola,Canada,580700,580700,512600,885
6,6,5,1965-12-31,Canola,Saskatchewan,224600,224600,242700,1080
7,7,6,1965-12-31,Rye,Alberta,81000,81000,116400,1435
8,8,7,1965-12-31,Rye,Canada,323900,323900,453400,1400
9,9,8,1965-12-31,Rye,Saskatchewan,166000,166000,224000,1350


## Problem 4

#### Which geographies are included in the farm prices dataset?


### Solution 4


In [ ]:
distinct_geographies_query <- "SELECT DISTINCT(GEO) FROM FARM_PRICES;"
distinct_geo <- sqlQuery(conn, distinct_geographies_query)
distinct_geo

## Problem 5

#### How many hectares of Rye were harvested in Canada in 1968?


### Solution 5


In [ ]:
sum_hectares_query <- "SELECT SUM(HARVESTED_AREA) FROM CROP_DATA WHERE YEAR = 1968 AND CROP_TYPE = 'Rye' AND GEO = 'Canada';"
sum_hectares <- sqlQuery(conn, sum_hectares_query)
sum_hectares 

## Problem 6

#### Query and display the first 6 rows of the farm prices table for Rye.


### Solution 6


In [ ]:
limit_rye_query <- "SELECT * FROM FARM_PRICES WHERE CROP_TYPE = 'Rye' LIMIT 6;"
limit_rye <- sqlQuery(conn, limit_rye_query)
limit_rye 

## Problem 7

#### Which provinces grew Barley?


### Solution 7


In [ ]:
barley_provinces_query <- "SELECT DISTINCT(GEO) FROM CROP_DATA WHERE CROP_TYPE = "Barley";"
barley_provinces <- sqlQuery(conn, barley_provinces_query)
barley_provinces

## Problem 8

#### Find the first and last dates for the farm prices data.


### Solution 8


In [ ]:
first_last_dates_query <- "SELECT MIN(PRICE_DATE) AS FIRST_DATE, MAX(PRICE_DATE) AS LAST_DATE FROM FARM_PRICES;"
first_last_dates <- sqlQuery(conn, first_last_dates_query)
first_last_dates 

## Problem 9

#### Which crops have ever reached a farm price greater than or equal to $350 per metric tonne?


### Solution 9


In [ ]:
expensive_crops_query <- "SELECT CROP_TYPE FROM FARM_PRICES WHERE PRICE >= 350;"
expensive_crops <- sqlQuery(conn, expensive_crops_query)
expensive_crops

## Problem 10

#### Rank the crop types harvested in Saskatchewan in the year 2000 by their average yield. Which crop performed best?


### Solution 10


In [ ]:
rank_crops_query <- "SELECT CROP_TYPE, AVG_YIELD, RANK() OVER (ORDER BY AVG_YIELD DESC) AS CROP_RANK FROM CROP_DATA WHERE YEAR(YEAR) = 2000 AND GEO = 'Saskatchewan';"
best_crop_query <- "SELECT CROP_TYPE, AVG_YIELD, RANK() OVER (ORDER BY AVG_YIELD DESC) AS CROP_RANK FROM CROP_DATA WHERE YEAR(YEAR) = 2000 AND GEO = 'Saskatchewan' LIMIT 1;"

rank_crops <- sqlQuery(conn, rank_crops_query)
best_crop <- sqlQuery(conn, best_crop_query)

rank_crops
best_crop



## Problem 11

#### Rank the crops and geographies by their average yield (KG per hectare) since the year 2000. Which crop and province had the highest average yield since the year 2000?


### Solution 11


In [ ]:
avgyield_rank_query <- "SELECT CROP_TYPE, GEO, AVG_YIELD, RANK() OVER (ORDER BY AVG_YIELD) AS YIELD_RANK FROM CROP_DATA WHERE YEAR(YEAR) >= 2000'"
avgyield_toprank_query <- "SELECT CROP_TYPE, GEO, AVG_YIELD, RANK() OVER (ORDER BY AVG_YIELD) AS YIELD_RANK FROM CROP_DATA WHERE YEAR(YEAR) >= 2000 LIMIT 1;"

avgyield_rank <- sqlQuery(conn, avgyield_rank_query)
avgyield_toprank <- sqlQuery(conn, avgyield_toprank)

avgyield_rank
avgyield_toprank


## Problem 12

#### Use a subquery to determine how much wheat was harvested in Canada in the most recent year of the data.


### Solution 12


In [ ]:
canada_total_wheat_query <- "SELECT SUM(HARVESTED_AREA) AS CANADA_TOTAL_WHEAT FROM CROP_DATA WHERE CROP_TYPE = 'Wheat' AND YEAR = (SELECT MAX(YEAR) FROM CROP_DATA)"
canada_total_wheat <- sqlQuery(conn, canada_total_wheat_query)
canada_total_wheat 


## Problem 13

#### Use an implicit inner join to calculate the monthly price per metric tonne of Canola grown in Saskatchewan in both Canadian and US dollars. Display the most recent 6 months of the data.


### Solution 13


In [ ]:
price_canola_cad_query <- SELECT DISTINCT(PRICE_DATE) * FXUSDCAD, P.DATE FROM MONTHLY_FX F, FARM_PRICES P 
WHERE CROP_TYPE = 'Canola' AND GEO = 'Saskatchewan'AND P.PRICE_DATE >= (SELECT MAX(P.PRICE_DATE) - 6 MONTHS FROM FARM_PRICES);

price_canola_usd_query <- SELECT DISTINCT(PRICE_DATE) FROM MONTHLY_FX F, FARM_PRICES P 
WHERE CROP_TYPE = 'Canola' AND GEO = 'Saskatchewan' AND P.PRICE_DATE >= (SELECT MAX(P.PRICE_DATE) - 6 MONTHS FROM FARM_PRICES);

price_canola_cad <- sqlQuery(conn, price_canola_cad_query)
price_canola_usd <- sqlQuery(conn, price_canola_usd)

price_canola_cad
price_canola_usd



## Author(s)

<h4> Jeff Grossman </h4>

## Contributor(s)

<h4> Rav Ahuja </h4>

## Change log

| Date       | Version | Changed by    | Change Description                                                                                          |
| ---------- | ------- | ------------- | ----------------------------------------------------------------------------------------------------------- |
| 2021-04-01 | 0.7     | Jeff Grossman | Split Problem 1 solution cell into multiple cells, fixed minor bugs |
| 2021-03-12 | 0.6     | Jeff Grossman | Cleaned up content for production |
| 2021-03-11 | 0.5     | Jeff Grossman | Moved more advanced problems to optional honours module |
| 2021-03-10 | 0.4     | Jeff Grossman | Added introductory and intermediate level problems and removed some advanced problems |
| 2021-03-04 | 0.3     | Jeff Grossman | Moved some problems to a new practice lab as prep for this assignment
| 2021-03-04 | 0.2     | Jeff Grossman | Sorted problems roughly by level of difficulty and relegated more advanced ones to ungraded bonus problems  |
| 2021-02-20 | 0.1     | Jeff Grossman | Started content creation                                                                                    |

## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
